# A Numba Mandelbrot Example (Fixed for Modern Python/Numba)

In [ ]:
import numpy as np
from numba import cuda
from matplotlib.pyplot import imshow, show
from timeit import default_timer as timer

In [ ]:
@cuda.jit(device=True)
def mandel(x, y, max_iters):
    """Device function that calculates if a point belongs to Mandelbrot set"""
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            return i
    return max_iters

In [ ]:
@cuda.jit
def mandel_kernel(min_x, max_x, min_y, max_y, image, iters):
    """CUDA kernel for Mandelbrot set calculation"""
    height = image.shape[0]
    width = image.shape[1]
    
    pixel_size_x = (max_x - min_x) / width
    pixel_size_y = (max_y - min_y) / height
    
    # Get the 2D thread position within the grid
    x, y = cuda.grid(2)
    
    # Make sure we stay within bounds
    if x < width and y < height:
        real = min_x + x * pixel_size_x
        imag = min_y + y * pixel_size_y
        image[y, x] = mandel(real, imag, iters)

In [ ]:
# Create a GPU array
gimage = np.zeros((1024, 1536), dtype=np.uint8)

# Configure the blocks and grid
threadsperblock = (16, 16)
blockspergrid_x = (gimage.shape[1] + threadsperblock[0] - 1) // threadsperblock[0]
blockspergrid_y = (gimage.shape[0] + threadsperblock[1] - 1) // threadsperblock[1]
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the computation
start = timer()
d_image = cuda.to_device(gimage)
mandel_kernel[blockspergrid, threadsperblock](-2.0, 1.0, -1.0, 1.0, d_image, 20)
result_image = d_image.copy_to_host()
dt = timer() - start

print(f"Mandelbrot created on GPU in {dt:.6f} s")

# Display the result
imshow(result_image)
show()